<a href="https://colab.research.google.com/github/qhfks27/rani01/blob/master/Ch5_Summary*.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ch5. A3C

A2C는 REINFORCE 알고리즘의 두 가지 단점(에피소드가 끝날 때가지 기다려야함, 그래디언트의 분산이 큼)을 개선한 알고리즘이다.  
####A2C 알고리즘의 해결되지 못한 문제점  
-정책과 가치함수 학습시 사용하는 샘플 간의 시간적으로 높은 상관관계 <br> $\rightarrow$ 그래디언트 편향시키고 학습의 불안정 유발  
#### $\Rightarrow$ 이 문제를 개선하기 위한 방법 : 비동기 A2C(A3C)

앞의 어드밴티지 액터-크리틱 목적함수 그래디언트 식을 한 개의 에피소드만 고려하여 근사하면 다음과 같다.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T\nabla_\theta log\pi_\theta(u_t \mid x_t)A^{\pi_\theta}(x_t, u_t))$$

<br>

배치 경사상승법을 통한 액터 신경망 파라미터 업데이트는 다음과 같다.

<br>

$$\theta \leftarrow \theta + \alpha\sum_{t=0}^T\nabla_\theta log\pi_\theta(u_t \mid x_t)A^{\pi_\theta}(x_t, u_t))$$

<br>

![1](https://user-images.githubusercontent.com/52277776/85367260-e6fdbd80-b563-11ea-8d42-a4450ed7ca96.jpg)

<br>

일반적인 미니배치 데이터로 업데이트하려면 무작위로 배치의 데이터를 섞은 후 파라미터를 반복 계산한다.

<br>

![2](https://user-images.githubusercontent.com/52277776/85367268-e9f8ae00-b563-11ea-8edc-b4659c7c6b30.jpg)

<br>

#### A2C에서도 이와 유사하게, 일정 시간 N 동안 다음과 같이 파라미터를 업데이트 했다.

<br>

$$\theta \leftarrow \theta + \alpha\sum_{t=i}^{i+N-1}\nabla_\theta log\pi_\theta(u_t \mid x_t)A^{\pi_\theta}(x_t, u_t))$$

<br>

형식적으로는 일반적인 미니배치 업데이트와 같지만, 내용적으로는 다르다.

#### A2C 에서는 에이전트가 기존 정책으로 일정 시간스텝 N동안 수집한 N개의 샘플[$(x_i, u_i, r(x_i, u_i), x_{i+1}$)]을 얻은 후 정책을 새롭게 업데이트 한다.

<br>

![3](https://user-images.githubusercontent.com/52277776/85367271-ebc27180-b563-11ea-9a7b-5df588a02838.jpg)

<br>

#### 확률적 경사하강법을 이용한 최적화 방법에서 기본 가정은 학습 데이터가 독립적이고 동일한 분포를 가진 샘플이어야 한다는 것이다.

하지만 A2C에서는 에이전트가 환경과 상호작용하면서 순차적으로 데이터를 얻기에, 에이전트의 샘플 $(x_i, u_i, r(x_i, u_i), x_{i+1}$) 도 강한 상관관계를 가져, 독립적이라고 볼 수 없다.  
 또한, 매 정책은 파라미터로 업데이트 되기 때문에 새로 얻은 데이터는 이전 데이터와는 다른 확률분포를 가질 것이다.

<br>

#### 상관관계 문제 해결을 위해 오프-폴리시(off-policy) 방법인 심층 Q 네트워크(DQN)에서는 에이전트의 데이터를 리플레이 버퍼(replay buffer)라는 곳에 저장해 두고, 어느 정도 모이면 무작위로 꺼내어 사용한다.

아래는 온-플리시 방법인 A2C에서 상관관계를 줄이기 위해 샘플링 기법을 이용한 목적함수 그래디언트의 근사식이다.

<br>

$$\nabla_\theta J(\theta)=\frac{1}{M}\sum_{M=1}^M\sum_{i=t}^{t+N-1}\nabla_\theta log\pi_\theta(u_i^{(m)} \mid x_i^{(m)})A^{\pi_\theta}(x_i^{(m)}, u_i^{(m)}))$$

<br>

#### 위 식에 의하면 동일한 정책으로 여러 개의 독립적 에피소드를 발생시킨 후, 로그-정책 그래디언트와 어드밴티지의 곱을 계산하여 모두 더하고 그에 대해 평균을 낸다.

( <i>그래디언트와 어드밴티지 곱을 계산하여 평균을 낸 값들로 순차적으로 진행하는 것과는 순서가 뒤집힌 개념이라고 할 수 있다.</i> )

#### 이 방법은 즉, 다중 에이전트를 병렬적으로 운용하여 각각의 데이터를 모으는 것이다.

<br>



기존의 어드밴티지 A2C는 동기적 A2C, 위에서 설명한 방식의 A2C는 비동기적 A2C라고 할 수 있고, 그 차이를 그림으로 나타내면 다음과 같다.

<br>

![4](https://user-images.githubusercontent.com/52277776/85367282-efee8f00-b563-11ea-9acd-12b89dd7fb67.jpg)

![5](https://user-images.githubusercontent.com/52277776/85367287-f250e900-b563-11ea-9b4c-33ec0ea4103b.jpg)


<br>

#### 비동기 A2C에서는 글로벌 신경망과 워커(worker)라고 불리는 에이전트가 있다. 둘은 각각의 액터 신경망과 크리틱 신경망을 가지고, 정보를 주고받으며 정책을 업데이트한다.

#### 비동기 방식에는 어떤 정보를 글로벌 신경망에 전달하고 어디서 목적함수 그래디언트를 계산하는지에 따라 두 가지 접근이 있다.

<br>

1. 그래디언트 병렬화(글로벌 신경망 계산량 감소, 워커의 수 확장 쉬움)<br>
 (1) 워커가 직접 자신의 그래디언트 계산 후 글로벌 신경망에 전달<br>
 (2) 글로벌 신경망이 자신의 신경망 업데이트 후 에이전트에 복사

<br>

![6](https://user-images.githubusercontent.com/52277776/85367290-f3821600-b563-11ea-80ba-3ced3d1baa5e.jpg)

<br>

2. 데이터 병렬화(워커는 데이터 수집만 수행)<br>
 (1) 워커가 샘플을 글로벌 신경망에 전달<br>
 (2) 글로벌 신경망이 목적함수 그래디언트 계산하여 신경망 업데이트 후 전달

<br>

![7](https://user-images.githubusercontent.com/52277776/85367293-f41aac80-b563-11ea-919e-1c8f6f196add.jpg)

<br>

#### 이 두가지 방법을 비동기 어드밴티지 액터-크리틱, A3C라고 한다. 
#### A3C에서는 이 밖에도 n-스텝 가치함수 추정 방법과 엔트로피 보너스 방법을 도입하였다. 

<br>

---


앞에서 언급하였듯이 목적함수 그래디언트 계산에서 편향 없이 작은 분산을 갖도록 추정하는 것이 중요하다.

 A2C 에서는 <b>1-스텝 관계식</b>을 이용해 가치함수 근사 식과 어드밴티지를 다음과 같이 추정하였다.

$$V_\phi(x_t)\approx r(x_t,u_t)+\gamma V_\phi(x_{t+1})$$

$$A_\phi (x_i,u_i)\approx r(x_i,u_i)+\gamma V_\phi(x_{i+1})-V_\phi(x_i)$$

<br>

 1-스텝 관계식을 이용하면 $A_\phi$ 의 분산은 작지만, $V_\phi$에 따라 $A_\phi$ 에 큰 평향이 있을 수 있다.

<br>

 반면, <b>몬테카를로 방식*</b>으로 다음과 같이 계산한다면 추정값에 편향은 없지만, 보상이 계속 누적되기 때문에 큰 분산을 갖게 된다.

*( <i>무작위 추출 난수를 이용하여 함수 값 계산</i> )

$$A_\phi (x_i,u_i)\approx \sum_{t=0}^{\infty}\gamma^{t-k}r(x_k,u_k)-V_\phi(x_t)$$

<br>

![8](https://user-images.githubusercontent.com/52277776/85367295-f54bd980-b563-11ea-9c58-a720badea4be.jpg)

<br>

#### 이 방식과 기존 방식의 중간을 취하는 방식이 다음과 같은 n-스텝 계산 방법이다.

$$V_\phi(x_t)\approx r(x_t,u_t)+\gamma r(x_{t+1}, u_{t+1})+\cdots+\gamma^{n-1}r(x_{t+n-1}, u_{t+n-1})+\gamma^n V_\phi(x_{t+n})$$

$$A_\phi (x_i,u_i)\approx r(x_t,u_t)+\gamma r(x_{t+1}, u_{t+1})+\cdots+\gamma^{n-1}r(x_{t+n-1}, u_{t+n-1})+\gamma^n V_\phi(x_{t+n})-V_\phi(x_t)=\sum_{k=t}^{t+n-1}\gamma^{t-k}r(x_k, u_k)+\gamma^n V_\phi(x_{t+n})-V_\phi(x_t)$$

<br>

#### n이 크면 분산은 커지고 편향이 작아지는 반면, n이 작으면 분산은 작아지고 편향이 커질 수 있다.

<br>

---

강화학습의 목표는 반환값의 기댓값인 목적함수를 최대화 하는 것이다.

이 표준 목적함수를 확장시켜 확률적 정책 에트로피를 추가한 새로운 형태를 <b>엔트로피 목적함수</b>라고 한다.

$$J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T\gamma^tr(x_t,u_t)]+\beta\sum_{t=0}^TE_{x_t \sim p_\theta(x_t)}[\gamma^t H(\pi_\theta(u_t \mid x_t)]$$

<br>

확률적 정책의 엔트로피 H는 다음과 같다.

$$H(\pi_\theta(u_t \mid x_t)=-\int\pi_\theta(u_t \mid x_t)log\pi_\theta(u_t \mid x_t)du_t$$

<br>

파라미터 $\beta$는 엔트로피와 반환값의 상대적 중요성을 결정한다.  
- 엔트로피 목적함수에 의하면 기존 반환값과 함께 정책의 엔트로피도 동시에 최대화 시킨다는 의도가 있다.  
- H는 확률 분포가 균일할 때 최댓값을 가지므로,<br>목적함수에 이를 더함으로써 에이전트가 최대한 무작위로 움직이게 하려는 목적이 있다.

<br>

---

#### 그래디언트 병렬화 A3C 알고리즘

 1. 글로벌 크리틱과 액터 신경망의 파라미터 $\phi$와 $\theta$ 초기화
 2. 다중 에이전트(워커) 생성 후 워커별로 $\phi_w$와 $\theta_w$를 글로벌 신경망에 동기화
 3. 워커별 반복  
    1. $t_{max}$ 시간스텝 동안 반복  
        1. 각 워커의 정책으로 행동 확률적으로 선택
        2. 행동의 보상과 다음 상태변수 측정
        3. 샘플 저장
    2. 워커의 n-스탭 시간차 타깃 계산 후 어드밴티지 계산  
    3. 워커 크리틱 신경망 그래디언트 계산
$$\sum_{i=1}[(y_{w,i}-V_{\phi_w}(x_i)\nabla_{\phi_w}V_{\phi_w}(x_i)]$$
    4. 워커 액터 신경망 그래디언트 계산
$$\nabla_{\theta_w}\sum_i(log\pi_{\theta_w}(u_t \mid x_t)A_{\phi_w}(x_t, u_t)+\beta H(\pi_{\theta_w}(x_t \mid u_t)))$$
    5. 글로벌 신경망으로 워커의 그래디언트 송부
    6. 워커의 그래디언트로 글로벌 신경망 업데이트
$$\phi \leftarrow \phi + \alpha_{critic}\sum_{i=1}[(y_{w,i}-V_{\phi_w}(x_i)\nabla_{\phi_w}V_{\phi_w}(x_i)]$$
$$\theta \leftarrow \theta + \alpha_{actor}\nabla_{\theta_w}\sum_i(log\pi_{\theta_w}(u_t \mid x_t)A_{\phi_w}(x_t, u_t)+\beta H(\pi_{\theta_w}(x_t \mid u_t)))$$  
    7. 업데이트된 글로벌 신경망 파라미터를 워커로 복사
 
<br>

#### 데이터 병렬화 A3C의 알고리즘은 위의 과정에서 워커가 샘플을 글로벌 신경망으로 전달하고, 글로벌 신경망은 그 샘플을 통해 업데이트되는 과정으로 진행된다.  

<br>

#### 데이터 병렬화 A3C 알고리즘
 1. 글로벌 크리틱과 액터 신경망의 파라미터 $\phi$와 $\theta$ 초기화
 2. 다중 에이전트(워커) 생성 후 워커별로 $\phi_w$와 $\theta_w$를 글로벌 신경망에 동기화
 3. 워커별 반복  
    1. $t_{max}$ 시간스텝 동안 반복  
        1. 각 워커의 정책으로 행동 확률적으로 선택
        2. 행동의 보상과 다음 상태변수 측정
        3. 샘플 저장
    2. $t_{max}$개의 샘플을 글로벌 신경망으로 송부
    3. 글로벌 신경망에서 n-스탭 시간차 타깃 계산 후 어드밴티지 계산  
    4. 글로벌 크리틱 신경망, 글로벌 액터 신경망 업데이트
$$\phi \leftarrow \phi + \alpha_{critic}\sum_{i=1}[(y_{i}-V_{\phi}(x_i)\nabla_{\phi}V_{\phi}(x_i)]$$
$$\theta \leftarrow \theta + \alpha_{actor}\nabla_{\theta}\sum_i(log\pi_{\theta}(u_t \mid x_t)A_{\phi}(x_t, u_t)+\beta H(\pi_{\theta}(x_t \mid u_t)))$$  
    7. 업데이트된 글로벌 신경망 파라미터를 워커로 복사